In [ ]:
srun -G1 --pty bash -c "source /data/ai_club/team_3_2024-25/team3-env-finetune/bin/activate; \
    hostname; \
    jupyter notebook \
        --ServerApp.root_dir=$(pwd) \
        --ServerApp.password='' \
        --ServerApp.open_browser=False \
        --ServerApp.allow_origin='*' \
        --ServerApp.allow_remote_access=True \
        --ServerApp.port=14321 \
        --ServerApp.ip='*'
"

In [1]:
import torch
import torch.nn as nn
from torch.distributions import Categorical
from transformers.models.qwen2.modeling_qwen2 import Qwen2DecoderLayer
from transformers import Qwen2ForCausalLM, Qwen2Config
import transformers
# import json
# import gc

2025-04-11 18:49:24.051897: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-11 18:49:24.064726: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744411764.080614  997783 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744411764.085476  997783 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-11 18:49:24.101730: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [33]:
class IMDecoderLayer(nn.Module):
    mask = None
    vspace_to_emb = None
    emb_to_vspace = None
    block_strength = []

    # scratch = None
    # norm = None

    def __init__(self, original_layer, emb_to_vspace, vspace_to_emb, norm, config, block_idx):
        super().__init__()
        self.original_layer = original_layer

        if IMDecoderLayer.vspace_to_emb == None:
            IMDecoderLayer.vspace_to_emb = vspace_to_emb

        if IMDecoderLayer.emb_to_vspace == None:
            IMDecoderLayer.emb_to_vspace =  emb_to_vspace

        # if IMDecoderLayer.scratch == None:
        #     IMDecoderLayer.scratch = torch.zeros(config.vocab_size, dtype=bool).to('cuda')

        # if IMDecoderLayer.norm == None:
        #     IMDecoderLayer.norm = norm

        self.block_idx = len(IMDecoderLayer.block_strength)
        IMDecoderLayer.block_strength.append(
            nn.Parameter(torch.tensor(1.0, dtype=torch.float32).to('cuda'))
        )

        self.vstate = torch.zeros(config.vocab_size).to('cuda')

    def forward(self, hidden_states, *args, **kwargs):
        hidden_states = self.original_layer(hidden_states, *args, **kwargs)
        hidden_states = hidden_states[0]

        mask = IMDecoderLayer.mask
        assert mask != None
        if mask:
            n_allowed = len(mask)
            n_disallowed = self.vstate.shape[-1] - n_allowed

            self.vstate *= 0
            self.vstate[:] = -1/n_disallowed
            self.vstate[mask] = 1/n_allowed
            
            hidden_states[-1,-1,:] += (self.vstate @ IMDecoderLayer.vspace_to_emb.weight) * IMDecoderLayer.block_strength[self.block_idx]

        # residual = hidden_states
        # hidden_states = IMDecoderLayer.emb_to_vspace(residual)
        
        # assert IMDecoderLayer.mask != None

        # # for i, positions in enumerate(IMDecoderLayer.mask):
        #     # for j, toks_allowed in enumerate(positions):
        # toks_allowed = IMDecoderLayer.mask
        # i, j = -1, -1

        # if toks_allowed:
        #     # hidden_states[i,j,:] = 0 

        #     IMDecoderLayer.scratch[:] = False
        #     IMDecoderLayer.scratch[toks_allowed] = True
        #     hidden_states[i,j,IMDecoderLayer.scratch] += 1/IMDecoderLayer.scratch.sum()

        #     IMDecoderLayer.scratch[:] = True
        #     IMDecoderLayer.scratch[toks_allowed] = False
        #     hidden_states[i,j,IMDecoderLayer.scratch] -= 1/IMDecoderLayer.scratch.sum()

        # # print(hidden_states)
        # hidden_states = hidden_states @ IMDecoderLayer.vspace_to_emb.weight
        # hidden_states = hidden_states * IMDecoderLayer.block_strength[self.block_idx]
        # hidden_states = hidden_states + residual

        return (hidden_states,)

def gen_mask(tokens):
    # mask = [] # mask[batch, position, allowed_tok]
    # for batch_size in tokens['attention_mask'].argmin(axis=1):
    #     if batch_size == 0:
    #         batch_size = tokens['attention_mask'].shape[1]
    #     mask.append([[] for i in range(batch_size)])

    mask = [] # mask[allowed_tok]

    return mask

for i, s in enumerate(IMDecoderLayer.block_strength):
    c=3 # This is a hyperparameter
    s.data.fill_(c*i/(i+c))

In [34]:
def get_model(do_masking):
    MODEL_NAME = 'Qwen/Qwen2.5-0.5B-Instruct'

    config = Qwen2Config.from_pretrained(MODEL_NAME)

    model = Qwen2ForCausalLM.from_pretrained(MODEL_NAME).to('cuda')
    tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAME)

    if do_masking:
        # FREEZE existing model. Only the new layer in IMDecoderLayer will be trained
        for param in model.parameters():
            param.requires_grad = False

        # REPLACE transformer blocks with IM ones
        for i, _ in enumerate(model.model.layers):
            model.model.layers[i] = IMDecoderLayer(model.model.layers[i], model.lm_head, model.model.embed_tokens, model.model.norm, config, i)

    def tokenize(batch):
        tokens = tokenizer(batch, return_tensors='pt', padding=True)
        tokens = {k:v.to('cuda') for k,v in tokens.items()}
        return tokens

    def tokof(s, check=True):
        toks = tokenizer(s, add_special_tokens=False)['input_ids']
        if check:
            if len(toks) > 1: raise Exception(f'This is more than one tok: {toks}')
            return toks[0]
        return toks

    return model, tokenize, tokenizer, tokof

model, tokenize, tokenizer, tokof = get_model(True)

In [137]:
# vocab = [
#     '.', '!', ',', ':',
#     'terve', 'hei', 'talo', 'vesi', 'ystävä', 'huomenta', 'velho', 'suomi', 'koira', 'nimi', 'nimeni', 'nimesi', 'nimensä',
#     'ystäväni', 'ystäväsi', 'ystävänsä', 'vanha', 'hyvää', 'suomalainen', 'mukava', 'minä', 'minun', 'olen', 'olenko', 'sinä', 'sinun', 'olet',
#     'oletko', 'hän', 'hänen', 'on', 'onko', 'matti', 'aleksi', 'sami', 'kyllä', 'ei', 'mitä', 'mikä', 'kuka', 'rossi', 'lucas'
# ]

vocab = [
    '.', '!', ',', ':',
    'hello', 'this', 'is', 'my', 'story', 'i', 'went', 'to', 'the', 'a', 'saw'
    'car', 'store', 'park', 'and', 'wizard', 'saw', 'buy', 'buying', 'oranges', 'apples', 'stuff', 'good', 'wizards', 'am'
]

vocab_raw = vocab.copy()

vocab += [v[0].upper() + v[1:] for v in vocab]
vocab += [(' '+v if v.isalpha() else v) for v in vocab]
# vocab += [v+'.' for v in vocab]

vocab = list(set(vocab))

# --- BUILD DA TRIE ---

trie = {}

for v in vocab:
    curr_node = trie

    toks = tokof(v, check=False)

    for tok in toks:
        tok = tokenizer.decode(tok) # FOR VISUALIZING
        if tok not in curr_node:
            curr_node[tok] = {}
        curr_node = curr_node[tok]

    curr_node[None] = {}

def get_next_allowed(given, trie):
    allowed = trie
    for tok in given:
        if tok in allowed:
            allowed = allowed[tok]
        elif None in allowed and tok in trie:
            allowed = trie[tok]
        else:
            # raise Exception(f'Unexpected token {tok}')
            given = ['.'] # NOTE: fix for invalid prior seq - just pretend we're starting a new word

    allowed = list(allowed.keys())

    if None in allowed and given:
        allowed += [t for t in trie.keys()]
        # allowed += [t for t in trie.keys() if t[0] == ' ' or not t.isalpha()]

    allowed = [v for v in allowed if v]

    # if not given:
    #     allowed = [v for v in allowed if v[0] != ' ']

    return allowed
    
# print(
    # get_next_allowed([], trie),
    # get_next_allowed([' o'], trie),
    # get_next_allowed([' o', 'len'], trie),
    # get_next_allowed([' o', 'len', 'ko'], trie),
    # get_next_allowed([' o', 'len', 'ko', ' hu'], trie),
    # get_next_allowed([' o', 'len', 'ko', ' hu', 'oment'], trie),
    # get_next_allowed([' o', 'len', 'ko', ' hu', 'oment', 'a'], trie),
    # get_next_allowed([' on', 'ok', 'ko', ' O'], trie),
#     sep='\n\n'
# )

In [147]:
# prompt = 'Here\'s a Finnish sentence: Onko'
prompt = f'''<|im_start|>system
Your responses can only draw from this allowed vocab: {vocab_raw}.<|im_end|>
<|im_start|>user
Tell me a short story with only the allowed vocab<|im_end|>
<|im_start|>assistant
'''

for i, s in enumerate(IMDecoderLayer.block_strength):
    c=7 # This is a hyperparameter
    s.data.fill_(c*i/(i+c))

print(prompt, end='')

for i in range(50):
    tokens = tokenize(prompt)

    # TODO: for now, only mask last token -- pros: simper mask, faster inference. cons: cant cannot parallelize training (doesn't matter for now)

    given = [tokenizer.decode(t) for t in tokens['input_ids'][0]]
    allowed = get_next_allowed(given, trie)
    # print(allowed)
    allowed = [tokof(t) for t in allowed] + [tokof('<|im_end|>')]

    IMDecoderLayer.mask = gen_mask(tokens)
    IMDecoderLayer.mask += allowed
    # IMDecoderLayer.mask[0][-1] += allowed

    out = model.generate(
        **tokens,
        max_new_tokens=1,
        pad_token_id=tokenizer.eos_token_id,
        # temperature=0.2,
        temperature=0.00001,
        # do_sample=True,
        return_dict_in_generate=True,
        # output_hidden_states=True
        output_logits=True
    )
    
    logits = out.logits[0][0]
    logits[allowed] += 10/(i+1)**2
    # tok_id = Categorical(logits=logits).sample() # temp = 1
    tok_id = logits.argmax()
    tok = tokenizer.decode(tok_id)

    # tok = tokenizer.decode(out.sequences[0][-1])

    prompt += tok
    print(tok, end='')

<|im_start|>system
Your responses can only draw from this allowed vocab: ['.', '!', ',', ':', 'hello', 'this', 'is', 'my', 'story', 'i', 'went', 'to', 'the', 'a', 'sawcar', 'store', 'park', 'and', 'wizard', 'saw', 'buy', 'buying', 'oranges', 'apples', 'stuff', 'good', 'wizards', 'am'].<|im_end|>
<|im_start|>user
Tell me a short story with only the allowed vocab<|im_end|>
<|im_start|>assistant
Hello, this is my story, I went to the store, I saw a car, I saw a park, and I saw a wizard, I saw a wizard, I saw an orange, and an apple, and I bought oranges and apples,